In [0]:
# Lendo o arquivo CSV com Spark
df = spark.read.csv('/FileStore/tables/vendas.csv', header=True, inferSchema=True)

# Verificando o esquema do DataFrame
df.printSchema()

# Mostrando as primeiras linhas para validar o formato dos dados
df.show(10)


root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

+--------------------+------------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|     Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+------------+-------

In [0]:
# Importando as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Criando uma Spark session
spark = SparkSession.builder \
    .appName("Processamento de Vendas") \
    .getOrCreate()

In [0]:
# Lendo o arquivo CSV
file_path = '/FileStore/tables/vendas.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Verificando o esquema do DataFrame
df.printSchema()

# Mostrando as primeiras linhas para validar o formato dos dados
df.show(5)


root
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Item Type: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- Order Priority: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Units Sold: integer (nullable = true)
 |-- Unit Price: double (nullable = true)
 |-- Unit Cost: double (nullable = true)
 |-- Total Revenue: double (nullable = true)
 |-- Total Cost: double (nullable = true)
 |-- Total Profit: double (nullable = true)

+--------------------+-------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------+---------------+------

In [0]:
# Limpando e preparando os dados
df_clean = df.dropna(subset=["Order Date", "Item Type", "Units Sold", "Unit Price", "Sales Channel"])

# Convertendo as colunas para o tipo apropriado
df_clean = df_clean.withColumn("Units Sold", df_clean["Units Sold"].cast("int")) \
                   .withColumn("Unit Price", df_clean["Unit Price"].cast("float"))

# Identificando o produto mais vendido em termos de quantidade e canal
produto_mais_vendido_canaldf = df_clean.groupBy("Item Type", "Sales Channel").agg(F.sum("Units Sold").alias("total_vendido"))
produto_mais_vendido_canaldf = produto_mais_vendido_canaldf.orderBy(F.desc("total_vendido"))
produto_mais_vendido_canaldf.show(1)  # Mostra o produto mais vendido e o canal


+---------+-------------+-------------+
|Item Type|Sales Channel|total_vendido|
+---------+-------------+-------------+
|   Cereal|      Offline|   1044443977|
+---------+-------------+-------------+
only showing top 1 row



In [0]:
# Calculando o valor da venda
df_clean = df_clean.withColumn("valor_venda", F.col("Units Sold") * F.col("Unit Price"))

# Agrupando por país e região
vendas_por_pais_regiao = df_clean.groupBy("Country", "Region").agg(F.sum("valor_venda").alias("total_vendas"))

# Ordenando para encontrar o país e região com maior volume de vendas
vendas_por_pais_regiao = vendas_por_pais_regiao.orderBy(F.desc("total_vendas"))
vendas_por_pais_regiao.show(1)  # Mostra o país e a região com maior volume de vendas

+-------+------------------+--------------------+
|Country|            Region|        total_vendas|
+-------+------------------+--------------------+
| Rwanda|Sub-Saharan Africa|3.634388989878637E10|
+-------+------------------+--------------------+
only showing top 1 row



In [0]:
# Extraindo mês e ano da coluna Order Date
df_clean = df_clean.withColumn("Order Date", F.to_date("Order Date", "MM/dd/yyyy")) \
                   .withColumn("Year", F.year("Order Date")) \
                   .withColumn("Month", F.month("Order Date"))

# Calculando as vendas mensais por produto
media_vendas_mensais = df_clean.groupBy("Item Type", "Year", "Month").agg(F.sum("valor_venda").alias("total_vendas_mensais"))
media_vendas_mensais = media_vendas_mensais.groupBy("Item Type").agg(F.avg("total_vendas_mensais").alias("media_mensal"))

# Exibindo as médias de vendas mensais por produto
media_vendas_mensais.show()

+---------------+--------------------+
|      Item Type|        media_mensal|
+---------------+--------------------+
|      Baby Food|4.1219792454741373E9|
|         Cereal| 3.321895262874927E9|
|           Meat|  6.81605948530933E9|
|      Household|1.079328847352320...|
|     Vegetables|2.4874873756124144E9|
|      Beverages| 7.664228833949307E8|
|Office Supplies|1.052364882462699...|
|      Cosmetics| 7.061430971496025E9|
|  Personal Care|1.3192387403278487E9|
|         Fruits|1.5059955350336576E8|
|         Snacks|2.4645035610805306E9|
|        Clothes|1.7648439439181597E9|
+---------------+--------------------+



In [0]:
# Resultados
print("1. Produto mais vendido:")
produto_mais_vendido = produto_mais_vendido_canaldf.first()
print(f"Produto: {produto_mais_vendido['Item Type']}, Canal: {produto_mais_vendido['Sales Channel']}, Total vendido: {produto_mais_vendido['total_vendido']} unidades")

if 'Country' in df_clean.columns:
    regiao_pais_maior_vendas = vendas_por_pais_regiao.first()
    print("\n2. País e região com maior volume de vendas:")
    print(f"País: {regiao_pais_maior_vendas['Country']}, Região: {regiao_pais_maior_vendas['Region']}, Total de vendas: R${regiao_pais_maior_vendas['total_vendas']:.2f}")

print("\n3. Média de vendas mensais por produto:")
for row in media_vendas_mensais.collect():
    print(f"Produto: {row['Item Type']}, Média mensal: R${row['media_mensal']:.2f}")


1. Produto mais vendido:
Produto: Cereal, Canal: Offline, Total vendido: 1044443977 unidades

2. País e região com maior volume de vendas:
País: Rwanda, Região: Sub-Saharan Africa, Total de vendas: R$36343889898.79

3. Média de vendas mensais por produto:
Produto: Baby Food, Média mensal: R$4121979245.47
Produto: Cereal, Média mensal: R$3321895262.87
Produto: Meat, Média mensal: R$6816059485.31
Produto: Household, Média mensal: R$10793288473.52
Produto: Vegetables, Média mensal: R$2487487375.61
Produto: Beverages, Média mensal: R$766422883.39
Produto: Office Supplies, Média mensal: R$10523648824.63
Produto: Cosmetics, Média mensal: R$7061430971.50
Produto: Personal Care, Média mensal: R$1319238740.33
Produto: Fruits, Média mensal: R$150599553.50
Produto: Snacks, Média mensal: R$2464503561.08
Produto: Clothes, Média mensal: R$1764843943.92
